# Setting up the YT API

In [1]:
import pandas as pd

In [31]:
# import file with all cleaned yt channel_ids
df = pd.read_csv('socials_nicks')

In [25]:
df.head(10)

,username,Twitter,YouTube,Instagram,Discord
0,tommyinnit,tommyinnit,NaN,tommyinnitt,https://discord.gg/an6rt4d
1,Quackity,quackity,UCFbZ2e9IrPejOdp8wsKUxvA,?hl=en,https://www.discord.gg/quackity
2,Sapnap,sapnap,UCEKH2P285Qylr60_e32Xn_g,sapnapinsta,https://discord.gg/qwmnrnp
3,auronplay,auronplay?ref_src=twsrc%5egoogle%7ctwcamp%5ese...,UCFR2oaNj02WnXkOgLH0iqOA,auronplay,NaN
4,xQcOW,xqc,UCmDTrq0LNgPodDOFZiSbsww,NaN,https://discord.gg/xqcow
5,AdinRoss,adinross,UCpq0Ou2ObQr-LbTB6hsl8BA,adin.ross,NaN
6,WilburSoot,NaN,NaN,NaN,NaN
7,ibai,ibaillanos,UCaY_-ksFSQtTGk0y1HA_3YQ,ibaillanos,NaN
8,Tubbo,tubbolive,UCAz5JW1_oryewk0eR-eP7Bw,NaN,https://discord.gg/tubbo
9,LCK_Korea,NaN,UCw1DsweY9b2AKGjV4kGJP1A,?hl=ko,NaN


In [33]:
# define class to retrieve all the information per channel_id (plus an api_key you get from gcs)
# this class requires you to install !pip install python-youtube

import json
import requests
from tqdm import tqdm


class YTstats:

    def __init__(self, api_key, channel_id):
        self.api_key = api_key
        self.channel_id = channel_id
        self.channel_statistics = None
        self.video_data = None

    def extract_all(self):
        self.get_channel_statistics()
        self.get_channel_video_data()

    def get_channel_statistics(self):
        """Extract the channel statistics"""
        print('get channel statistics...')
        url = f'https://www.googleapis.com/youtube/v3/channels?part=statistics&id={self.channel_id}&key={self.api_key}'
        pbar = tqdm(total=1)
        
        json_url = requests.get(url)
        data = json.loads(json_url.text)
        try:
            data = data['items'][0]['statistics']
        except KeyError:
            print('Could not get channel statistics')
            data = {}

        self.channel_statistics = data
        pbar.update()
        pbar.close()
        return data

    def get_channel_video_data(self):
        "Extract all video information of the channel"
        print('get video data...')
        channel_videos, channel_playlists = self._get_channel_content(limit=50)

        parts=["snippet", "statistics","contentDetails", "topicDetails"]
        for video_id in tqdm(channel_videos):
            for part in parts:
                data = self._get_single_video_data(video_id, part)
                channel_videos[video_id].update(data)

        self.video_data = channel_videos
        return channel_videos

    def _get_single_video_data(self, video_id, part):
        """
        Extract further information for a single video
        parts can be: 'snippet', 'statistics', 'contentDetails', 'topicDetails'
        """

        url = f"https://www.googleapis.com/youtube/v3/videos?part={part}&id={video_id}&key={self.api_key}"
        json_url = requests.get(url)
        data = json.loads(json_url.text)
        try:
            data = data['items'][0][part]
        except KeyError as e:
            print(f'Error! Could not get {part} part of data: \n{data}')
            data = dict()
        return data

    def _get_channel_content(self, limit=None, check_all_pages=True):
        """
        Extract all videos and playlists, can check all available search pages
        channel_videos = videoId: title, publishedAt
        channel_playlists = playlistId: title, publishedAt
        return channel_videos, channel_playlists
        """
        url = f"https://www.googleapis.com/youtube/v3/search?key={self.api_key}&channelId={self.channel_id}&part=snippet,id&order=date"
        if limit is not None and isinstance(limit, int):
            url += "&maxResults=" + str(limit)

        vid, pl, npt = self._get_channel_content_per_page(url)
        idx = 0
        while(check_all_pages and npt is not None and idx < 10):
            nexturl = url + "&pageToken=" + npt
            next_vid, next_pl, npt = self._get_channel_content_per_page(nexturl)
            vid.update(next_vid)
            pl.update(next_pl)
            idx += 1

        return vid, pl

    def _get_channel_content_per_page(self, url):
        """
        Extract all videos and playlists per page
        return channel_videos, channel_playlists, nextPageToken
        """
        json_url = requests.get(url)
        data = json.loads(json_url.text)
        channel_videos = dict()
        channel_playlists = dict()
        if 'items' not in data:
            print('Error! Could not get correct channel data!\n', data)
            return channel_videos, channel_videos, None

        nextPageToken = data.get("nextPageToken", None)

        item_data = data['items']
        for item in item_data:
            try:
                kind = item['id']['kind']
                published_at = item['snippet']['publishedAt']
                title = item['snippet']['title']
                if kind == 'youtube#video':
                    video_id = item['id']['videoId']
                    channel_videos[video_id] = {'publishedAt': published_at, 'title': title}
                elif kind == 'youtube#playlist':
                    playlist_id = item['id']['playlistId']
                    channel_playlists[playlist_id] = {'publishedAt': published_at, 'title': title}
            except KeyError as e:
                print('Error! Could not extract data from item:\n', item)

        return channel_videos, channel_playlists, nextPageToken

    def dump(self):
        """Dumps channel statistics and video data in a single json file"""
        if self.channel_statistics is None or self.video_data is None:
            print('data is missing!\nCall get_channel_statistics() and get_channel_video_data() first!')
            return

        fused_data = {self.channel_id: {"channel_statistics": self.channel_statistics,
                              "video_data": self.video_data}}

        channel_title = self.video_data.popitem()[1].get('channelTitle', self.channel_id)
        channel_title = channel_title.replace(" ", "_").lower()
        filename = channel_title + '.json'
        with open(filename, 'w') as f:
            json.dump(fused_data, f, indent=4)
        
        print('file dumped to', filename)

In [27]:
# extract channel info and all videos with detailed info
def create_json_yt(channel_ids, API_KEY):
    count = 0
    for channel_id in channel_ids:
        count += 1
        print(f'getting channel: {channel_id}')
        if channel_id:
            try:
                yt = YTstats(API_KEY, channel_id)
                yt.extract_all()
                yt.dump()  # dumps to .json
                print(count)
            except:
                print(count)
                continue
                
def get_channel_info(channel_ids, API_KEY):
    count = 0
    for channel_id in channel_ids:
        count += 1
        if channel_id:
            try:
                yt = YTstats(API_KEY, channel_id)
                yt.get_channel_statistics()
                yt.dump()  # dumps to .json
                print(count)
            except:
                print(count)
                continue

In [34]:
# get your api key from gcs and put it here inside the '', the channel_ids are generated from the df
API_KEY = ''

channel_ids = df.YouTube.to_list()

In [35]:
# define your range you want to work with, an example could be
Ludwig = channel_ids[500:550]
Katherina = channel_ids[550:600]
Ale = channel_ids[600:650]
Simon = channel_ids[650:700]

In [ ]:
# no need to define the function output to a variable cause we are not returning anything here
# instead, it will dump the json file for each streamer with the correct name in your current directory


100%|██████████| 1/1 [00:00<00:00,  6.65it/s]

getting channel: nan
get channel statistics...
Could not get channel statistics
get video data...



0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00,  8.01it/s]

Error! Could not get correct channel data!
 {'error': {'code': 403, 'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'errors': [{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]}}
1
getting channel: UCWP0-l2o2A_zPqJd6cg0Hsg
get channel statistics...
Could not get channel statistics
get video data...



0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00,  8.11it/s]

...

100%|██████████| 1/1 [00:00<00:00,  8.89it/s]
0it [00:00, ?it/s]
  0%|          | 0/1 [00:00<?, ?it/s]

Could not get channel statistics
get video data...
Error! Could not get correct channel data!
 {'error': {'code': 403, 'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'errors': [{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]}}
246
getting channel: UCAe59EK-MWy0l8CVbD8AMpQ
get channel statistics...


100%|██████████| 1/1 [00:00<00:00,  8.69it/s]
0it [00:00, ?it/s]
  0%|          | 0/1 [00:00<?, ?it/s]

Could not get channel statistics
get video data...
Error! Could not get correct channel data!
 {'error': {'code': 403, 'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'errors': [{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]}}
247
getting channel: nan
get channel statistics...


100%|██████████| 1/1 [00:00<00:00,  8.01it/s]

In [36]:
# we may want to define subranges for our names as I guess roughly 10 calls are possible per day
create_json_yt(#put your name here[:10], API_KEY)

SyntaxError: unexpected EOF while parsing (<ipython-input-36-dedbd099b916>, line 2)

In [ ]:
#only channel info (not relevant as of now)
yt.get_channel_statistics()

# Get YT Channel info (focus on chapter 1)

In [ ]:
def get_channel_stats(API_KEY, channel_id):
    from filename import YTstats # import Class from above
    yt = YTstats(API_KEY, channel_id)
    yt.get_channel_statistics()
    yt.dump()  # dumps to .json

# Get comments from YT

In [ ]:
## this needs to be run and stored cold!

In [ ]:
from googleapiclient.discovery import build
  
def video_comments(video_id):
    '''Takes a youtube video id and retrieves comments for that video'''
    timeout = 10   # [seconds]

    timeout_start = time.time()

    # empty list for storing reply
    replies = []
    comment = ''
    # creating youtube resource object
    youtube = build('youtube', 'v3',
                    developerKey=api_key)
  
    # retrieve youtube video results
    video_response=youtube.commentThreads().list(
    part='snippet,replies',
    videoId=video_id
    ).execute()
    df = pd.DataFrame(data=[[comment, replies]], columns=['comments', 'replies'])
    # iterate video response
    while video_response and time.time() < timeout_start + timeout:
        
        # extracting required info
        # from each result object 
        for item in video_response['items']:
            
            # Extracting comments
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            df = df.append({'comments': pd.Series(data=comment)}, ignore_index=True)
            # counting number of reply of comment
            replycount = item['snippet']['totalReplyCount']
  
            # if reply is there
            if replycount>0:
                
                # iterate through all reply
                for reply in item['replies']['comments']:
                    
                    # Extract reply
                    reply = reply['snippet']['textDisplay']
                    
                    # Store reply is list
                    replies.append(reply)
                    df = df.append({'replies': pd.Series(data=replies)}, ignore_index=True)
                    
            # print comment with list of reply
            print(comment, replies, end = '\n\n')
            # empty reply list
            replies = []
        
        # Again repeat
        if 'nextPageToken' in video_response:
            video_response = youtube.commentThreads().list(
                    part = 'snippet,replies',
                    videoId = video_id
                ).execute()
        else:
            break
    return df
# Enter video id
video_id = "4oKiOli9kFU"

In [ ]:
## trying to automate the uploading of a file stored in json 
## and the extraction of the video_ids, to then run the above function for all videos

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import json
%matplotlib inline

with open("noway4u.json") as datafile:
    data = json.load(datafile)
    keys_list = list(data)
    key = keys_list[0]
    video_ids = list(data[key]['video_data'])
    for video in video_ids:
        temp = video_comments(video)
        df = pd.concat([df, temp], axis=1)

# not relevant (trying NLP)

In [ ]:
temp3 = video_comments(video_id)

In [ ]:
word_cloud_text = ' '.join(str(v) for v in listen)

In [ ]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import string
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
stop_words = set(stopwords.words('german'))

In [ ]:
def clean_text(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    text = "".join(word for word in text if not word.isdigit()).lower()
    word_tokens = word_tokenize(text)
    text = [w for w in word_tokens if not w in stop_words]
    return ' '.join(text)

In [ ]:
temp3["clean_comments"] = temp3.comments.apply(lambda x: str(x).strip("dtype objec"))

temp3["clean_comments"] = temp3.clean_comments.apply(lambda x: clean_text(x))

temp3["clean_comments"] = temp3.clean_comments.apply(lambda x: str(x).strip("dtype"))

comment_list = temp3.clean_comments.to_list()
word_cloud_text = ' '.join(str(v) for v in comment_list)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


# Tuned TFidfvectorizer
vec = TfidfVectorizer(ngram_range = (2,2), min_df=0.01, max_df = 0.05).fit(temp3.clean_comments)

vectors = vec.transform(temp3.clean_comments) # Transform text to vectors

sum_tfidf = vectors.sum(axis=0) # Sum of tfidf weighting by word

tfidf_list = [(word, sum_tfidf[0, idx]) for word, idx in     vec.vocabulary_.items()]  # Get the word and associated weight

sorted_tfidf_list =sorted(tfidf_list, key = lambda x: x[1], reverse=True)  # Sort

sorted_tfidf_list

In [ ]:

# General packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

# NLP packages
import nltk
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from wordcloud import WordCloud

# Modeling packages
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from pylab import rcParams
import warnings
warnings.filterwarnings("ignore")
rcParams['figure.figsize'] = 14, 6
plt.style.use('ggplot')

wordcloud = WordCloud(max_font_size=50, # Maximum font size for the largest word
                      max_words=20, # The maximum number of words
                      background_color="white", # Background color for the word cloud image
                      scale = 10, # Scaling between computation and drawing
                      width=800, # Width of the canvas
                      height=400 # Height of the canvas
                     ).generate(word_cloud_text)

plt.figure()
plt.imshow(wordcloud, 
           interpolation="bilinear") # to make the displayed image appear more smoothly
plt.axis("off")
plt.show()